In [1]:
from keras.models import Model
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np

import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Restricting GPU memory
os.environ['CUDA_VISIBLE_DEVICES']='0'
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
import dataloader, TNN, evaluation # Loading helper functions

In [4]:
dev2train = dataloader.creating_id_mapping()

# Getting i-vectors
trn_bl_ivector, trn_bg_ivector, dev_bl_ivector, dev_bg_ivector, tst_ivector = dataloader.get_ivectors()

# Loading labels for task 2
trn_bl_id, trn_bg_id, dev_bl_id, dev_bg_id, tst_id = dataloader.get_spk_ids()

# Making labels for task 1
trn_ivector = np.append(trn_bl_ivector, trn_bg_ivector,axis=0) # combining bg and bl speakers into a single vector
trn_trials = np.append( np.ones([len(trn_bl_ivector), 1]), np.zeros([len(trn_bg_ivector), 1]))
dev_ivector = np.append(dev_bl_ivector, dev_bg_ivector,axis=0) # combining bg and bl
dev_trials = np.append( np.ones([len(dev_bl_id), 1]), np.zeros([len(dev_bg_id), 1]))
tst_trials = dataloader.get_tst_trials()

# Models

## TNN model

In [5]:
model, anchor_input, encoded_anchor = TNN.create_TNN()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 600)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 600)          360600      anchor_input[0][0]               
                                                                 positive_input[0][0]       

## Cosine Similarity

In [6]:
def cosine_similarity(embedding, bl_ivector_embedding, labels):
    spk_mean, _ = dataloader.make_spkvec(bl_ivector_embedding,labels)
    # This accuracy calcultion is provided by MCE2018
    scores = spk_mean.dot(embedding.transpose())
    blscores = spk_mean.dot(bl_ivector_embedding.transpose()) # This will be used in normalization and task 2
    
    # Multi-target normalization
    mnorm_mu = np.mean(blscores,axis=1) 
    mnorm_std = np.std(blscores,axis=1)
    for iter in range(np.shape(scores)[1]):
        scores[:,iter]= (scores[:,iter] - mnorm_mu) / mnorm_std
    pred_scores = np.max(scores,axis=0)

    return pred_scores

# Prediction phase

## Getting TNN embeddings

In [7]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)
trained_model.load_weights('./task2_trainall/triplet_Dense600_87nd_err285_C0.5.hdf5')
transformed_trn_bl_ivector = trained_model.predict(trn_bl_ivector)
transformed_dev_ivector = trained_model.predict(dev_ivector)
transformed_trn_ivector = trained_model.predict(trn_ivector)
transformed_tst_ivector = trained_model.predict(tst_ivector)
transformed_dev_bl_ivector = trained_model.predict(dev_bl_ivector)


## Evaluating on dev set

In [8]:
predictions = cosine_similarity(transformed_dev_ivector, transformed_trn_bl_ivector, trn_bl_id)
err = dataloader.calculate_EER(dev_trials,predictions)

Top S detector EER is 0.94%


## Evaluating on testset

### Train on trainset only, test on testset

In [9]:
predictions = cosine_similarity(transformed_tst_ivector, transformed_trn_bl_ivector, trn_bl_id)
err = dataloader.calculate_EER(tst_trials,predictions)

Top S detector EER is 8.49%


### Train on trainset and devset, test on testset

In [10]:
# getting dev ids to train id conversion
dev_bl_id_along_trnset = []
for iter in range(len(dev_bl_id)):
    dev_bl_id_along_trnset.extend([dev2train[dev_bl_id[iter]]])

combined_embedding = np.append(transformed_trn_ivector,transformed_dev_bl_ivector,0)
combined_labels = np.append(trn_bl_id,dev_bl_id_along_trnset)
predictions = cosine_similarity(transformed_tst_ivector, combined_embedding, combined_labels)
err = dataloader.calculate_EER(tst_trials,predictions)

Top S detector EER is 9.06%


In [11]:
# 